## Imports

In [1]:
import os
import random
import cv2
import numpy as np
from tqdm import tqdm as progressbar
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torchvision.datasets import CIFAR100, CIFAR10


ModuleNotFoundError: No module named 'cv2'

## Datasets

In [ ]:
class CelebA(Dataset):
    ...

class CelebAMask(Dataset):
    def __init__(self, csv_file, root_dir, eval_file, targets, split='train', transform=None):

        self.attr = pd.read_csv(csv_file)
        self.eval_partition = pd.read_csv(eval_file)
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.targets = targets
        
        self.attr = self.attr[self.targets]
        self.attr = self.attr.replace(-1, 0)
        self.attr = self.attr.set_index('image_id')
        self.eval_partition = self.eval_partition.set_index('image_id')
        self.attr = self.attr.join(self.eval_partition)
        self.attr['image_id'] = self.attr.index
        
        if self.split == 'train':
            self.attr = self.attr.loc[self.attr['partition']==0]
            self.attr = self.attr.drop('partition', axis=1)
        else:
            self.attr = self.attr.loc[self.attr['partition']==2]
            self.attr = self.attr.drop('partition', axis=1)
            
        self.attr = self.attr[self.targets] 

    def __len__(self):
        return len(self.attr)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
    
        img_name = self.root_dir + self.attr.iloc[idx, 0]
        
        image = cv2.imread(img_name) / 255.0
        attr = self.attr.iloc[idx, 1:]
        attr = np.array([attr])
        attr = attr.astype('float')
        
        image = cv2.resize(image, dsize=(96, 96), interpolation=cv2.INTER_AREA)
        image = image.reshape(image.shape[2], image.shape[0], -1)
        sample = (torch.tensor(image).float(), attr)

        if self.transform:
            sample = self.transform(sample)

        return sample
    
class SubsetSequentialSampler(torch.utils.data.Sampler):
    r"""Samples elements sequentially from a given list of indices, without replacement.

    Arguments:
        indices (sequence): a sequence of indices
    """

    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return (self.indices[i] for i in range(len(self.indices)))
    
    def __len__(self):
        return len(self.indices)
    
class MyDataset(Dataset):
    def __init__(self, dataset_name, train_flag, targets = None, transf=None):
        self.dataset_name = dataset_name
        
        if self.dataset_name == "cifar10":
            self.cifar10 = CIFAR10('../input/cifar10', train=train_flag, 
                                    download=True, transform=transf)
            
        if self.dataset_name == "celeba":
            attr_file = '../input/celeba-dataset/list_attr_celeba.csv'
            eval_file = '../input/celeba-dataset/list_eval_partition.csv'
            path = '../input/celeba-dataset/img_align_celeba/img_align_celeba/'            
            self.celeba = CelebA(attr_file, path, eval_file, targets, split='train')  

    def __getitem__(self, index):
        if self.dataset_name == "cifar10":
            data, target = self.cifar10[index]
            
        if self.dataset_name == "celeba":
            data, target = self.celeba[index]
        
        return data, target, index

    def __len__(self):
        if self.dataset_name == "cifar10":
            return len(self.cifar10)
        
        if self.dataset_name == "celeba":
            return len(self.celeba)

class CityScapes(Dataset):
    ...

## PT4AL

In [ ]:
class BasicBlock(nn.Module):
    
    def __init__(self, in_planes, planes, expansion, stride=1):
        super(BasicBlock, self).__init__()
        self.expansion = expansion
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, expansion=1):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], 1, stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], 1, stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], 1, stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], 1, stride=2)
        
        self.linear1 = nn.Linear(512*expansion, 2)
        self.linear2 = nn.Linear(512*expansion, 2)
        self.linear3 = nn.Linear(512*expansion, 2)
        self.linear4 = nn.Linear(512*expansion, 2)
        self.linear5 = nn.Linear(512*expansion, 2)
        self.linear6 = nn.Linear(512*expansion, 2)
        self.linear7 = nn.Linear(512*expansion, 2)
        self.linear8 = nn.Linear(512*expansion, 2)
        self.linear9 = nn.Linear(512*expansion, 2)
        self.linear10 = nn.Linear(512*expansion, 2)
        self.linear11 = nn.Linear(512*expansion, 2)

    def _make_layer(self, block, planes, num_blocks, expansion, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, expansion, stride))
            self.in_planes = planes * expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out1 = self.layer1(out)
        out2 = self.layer2(out1)
        out3 = self.layer3(out2)
        out4 = self.layer4(out3)
        out = F.avg_pool2d(out4, 4)
        outf = out.view(out.size(0), -1)
        outt1 = self.linear1(outf)
        outt2 = self.linear2(outf)
        outt3 = self.linear3(outf)
        outt4 = self.linear4(outf)
        outt5 = self.linear5(outf)
        outt6 = self.linear6(outf)
        outt7 = self.linear7(outf)
        outt8 = self.linear8(outf)
        outt9 = self.linear9(outf)
        outt10 = self.linear10(outf)
        outt11 = self.linear11(outf)

        return [outt1, outt2, outt3, outt4, outt5, outt6, outt7, outt8, outt9, outt10, outt11], outf, [out1, out2, out3, out4]

def ResNet18(num_classes = 10, expansion = 1):
    return ResNet(BasicBlock, [2,2,2,2], num_classes, expansion)

## Training Loop

## Args